In [11]:
from __future__ import print_function

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
from bs4 import BeautifulSoup
import json
import requests
import pickle
import os
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import cv2
import pandas as pd

#import imutils
#from imutils.contours import sort_contours
#import matplotlib.pyplot as plt
#from sewar.full_ref import uqi, vifp, psnrb

import string    
import re

import time 
import base64
import pendulum
from datetime import datetime

import os.path
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [12]:
from unidecode import unidecode
import re
import selenium
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import os
import time
import json
import glob

import matplotlib.pyplot as plt
from PIL import Image

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

from tensorflow.keras.models import load_model
import imutils
from imutils.contours import sort_contours
from datetime import datetime

In [13]:
# FILE_COMMON_PATH = "./"

In [14]:
f = FILE_COMMON_PATH + "lib_common.ipynb"
%run $f

f = FILE_COMMON_PATH + "lib_db.ipynb"
%run $f

f = FILE_COMMON_PATH + "config.ipynb"
%run $f

f = FILE_COMMON_PATH + "lib_crawl.ipynb"
%run $f

f = FILE_COMMON_PATH + "lib_ftp.ipynb"
%run $f

In [36]:
class PredictCaptcha:
    def __init__(self, fmodel = CAPTCHA_TAX_MODEL):
        self.fn_model = load_model(fmodel)
        
    @staticmethod
    def remove_noise(fimg):
        def convert_jpg_(fimg):

            png_image = Image.open(fimg) if type(fimg) is str else fimg

            jpg_image = Image.new("RGB", png_image.size, (255, 255, 255))  # Creating a white background
            jpg_image.paste(png_image, (0, 0), png_image)
            jpeg_array = np.array(jpg_image)

            return jpeg_array

        image = cv2.imread(fimg, cv2.IMREAD_UNCHANGED) if type(fimg) is str and '.jpg' in fimg else \
                        convert_jpg_(fimg)
        kernel = np.ones((2, 2), np.uint8)
        dilated_img = cv2.dilate(image, kernel, iterations=1)

        return dilated_img

    @staticmethod
    def chop_multi_versions(img, num_chars, chop_wd = [0, -2, -4, 2, 4], chop_width=[0]):
        def find_first_position_(img):
            arr_pos = []

            for r in range(0, img.shape[0]):
                fwhite = None
                for c in range(0, img.shape[1]):
                    s = sum(img[r][c])
                    fwhite = c if s == 765 else fwhite            
                    if fwhite is not None and (s != 765):  
                        arr_pos.append(c) if c < img.shape[1] - 3 else None
                        break
            return min(arr_pos)

        def find_last_position_(img):
            arr_pos = []

            for r in range(0, img.shape[0]):
                fwhite = None
                for c in range(0, img.shape[1]):
                    c = img.shape[1] - c - 1
                    s = sum(img[r][c])            
                    fwhite = c if s == 765 else fwhite
                    if fwhite is not None and (s != 765):
                        arr_pos.append(c) if c > 5 else None
                        break

            return max(arr_pos)

        def chop_first_last_(img, first_chop, last_chop):
            return np.array([img[i][first_chop:last_chop] for i in range(0, img.shape[0])])

        def chop_equal_(img, num_chars):
            equal_size = int(img.shape[1]/num_chars)
            return [img[0:img.shape[0], i:i+equal_size,:] for i in range(0, equal_size*num_chars, equal_size)]


        def chop_multi_(img, num_chars, chop_wd = [0, -2, -4, 2, 4], chop_width=[0]):
            equal_size = int(img.shape[1]/num_chars)
            chop_multi = [[] for i in range(0, num_chars)]
            chop_size = [(x, x, w) for x in chop_wd for w in chop_width]        
            for i in range(0, num_chars):
                for s in chop_size:
                    ri = i*equal_size
                    start, end = max(0, ri + s[0]), min(img.shape[1], ri + equal_size + s[1] + s[2])

                    t_img = img[0:img.shape[0], start:end,:]

                    # padding if width less than equal width
                    num_padded = (equal_size + max(chop_width)) - (end - start)
                    num_left = int(num_padded / 2)
                    num_right = num_padded - num_left

                    padded_left = np.full((num_left, 3), 255) if num_padded > 0 else None
                    padded_right = np.full((num_right, 3), 255) if num_padded > 0 else None
                    t_img = np.array([np.concatenate((padded_left, t, padded_right), axis=0) 
                                              for t in t_img]) if num_padded > 0 else t_img

                    chop_multi[i].append(t_img)

            return chop_multi

        def padding_multi_(chop_multi, padded = np.full((2, 3), 255)):
            padded_multi = [[] for i in range(len(chop_multi))]
            for i in range(len(chop_multi)):
                for j in range(len(chop_multi[i])):
                    img = chop_multi[i][j]
                    t_img = np.array([np.concatenate((padded, t, padded), axis=0) for t in img])
                    padded_multi[i].append(t_img.astype(np.uint8))

            return padded_multi


        first_chop = find_first_position_(img)
        last_chop = find_last_position_(img)
        chop_image = chop_first_last_(img, first_chop, last_chop)    
        chop_list = chop_multi_(chop_image, num_chars, chop_wd, chop_width)    
        padding_list = padding_multi_(chop_list)

        return padding_list

    @staticmethod
    def extract_contours_all(img_list):
        def extract_contours_(img):
            img = imutils.resize(img, width=32, height=32)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            edged = cv2.Canny(blurred, img.shape[0], img.shape[1])
            # edged = cv2.Canny(img, img.shape[0], img.shape[1])
            cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            # cnts_sorted = sorted(cnts, key=lambda c: cv2.boundingRect(c)[0])
            cnts_sorted = cnts

            chars = []
            for i, c in enumerate(cnts_sorted):

                # compute the bounding box of the contour
                (x, y, w, h) = cv2.boundingRect(c)            

                if w >= 10 and h >= 10:
                    roi = edged[y:y + h, x:x + w]
                    thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
                    (tH, tW) = thresh.shape

                    # if the width is greater than the height, resize along the width dimension
                    rsw, rsh = 32, 32                
                    thresh = imutils.resize(thresh, width=rsw) if tW > tH else \
                                    imutils.resize(thresh, height=rsh)
                    (tH, tW) = thresh.shape
                    dX = int(max(0, rsw - tW) / 2.0)
                    dY = int(max(0, rsh - tH) / 2.0)

                    # pad the image and force 32x32 dimensions
                    padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY, left=dX, right=dX, 
                                                borderType=cv2.BORDER_CONSTANT,value=(0, 0, 0))
                    padded = cv2.resize(padded, (rsw, rsh))
                    padded = padded.astype("float32")
                    padded = np.expand_dims(padded, axis=-1)
                    chars.append((padded, (x, y, w, h)))

            chars = sorted(chars, key=lambda c: c[1][2], reverse=True)
            return chars


        def extract_contours_list_(img_list):
            chars_list = None
            for img in img_list:
                cts = extract_contours_(img)
                if len(cts) > 0:
                    chars_list = [cts[0]] if chars_list is None else chars_list + [cts[0]]
            # chars_list = sorted(chars_list, key=lambda c: c[1][2], reverse=True)

            return chars_list


        return extract_contours_list_(img_list)

    @staticmethod
    def process_char(img, pos, num_chars=5, chop_wd = [0, -2, -4, 2, 4], chop_width=[0]):
        img_rn = PredictCaptcha.remove_noise(img)
        chop_imgs = PredictCaptcha.chop_multi_versions(img_rn, num_chars, chop_wd, chop_width)
        all_chars = PredictCaptcha.extract_contours_all(chop_imgs[pos])

        return all_chars


    def predict(self, fimg):
        format_y = lambda y: ''.join(map(lambda x: chr(int(x)), y))
        res_chars = ''
        y_pred_ls = []
        y_pred_ls_all = []
        for pos in range(5):
            chars_img = self.process_char(fimg, pos, num_chars=5, chop_wd=[0, -2, 2], chop_width=[0, 2])
            y_pred_ls = []
            for cimg in chars_img:
                im = np.array([np.repeat(x, 3, axis=1) for x in cimg[0]], np.uint8)
                im = np.array(im) / 255.0
                y_pred = self.fn_model.predict(np.array([im]), verbose=0)
                am = tf.math.argmax(y_pred, axis=-1)
                c = format_y(am[0])
                y_pred_ls.append({'c': c, 'pc': y_pred[0][0][am][0][0]})

            dft = pd.DataFrame.from_records(y_pred_ls)        
            dft = dft.groupby('c')['pc'].agg(['count', 'max']).reset_index()\
                        .sort_values(by=['count'], ascending=False).head(2)

            # fix the ad hoc case r, h
            if set(dft['c']) == set(['r', 'h']):
                top_char = 'r'
            else: 
                top_char = dft.sort_values(by=['max'], ascending=False).head(1).iloc[0][0]

            y_pred_ls_all.append(y_pred_ls)

            res_chars += top_char

        return {
            'predict': res_chars, 
            'params': y_pred_ls_all
        }

In [58]:
from unidecode import unidecode
import re
import selenium

class CrawlTax():
            
    def __init__(self, search_type="tax_num", chrome_execute = None, chrome_download=None, 
                 driver=None, predict_captcha=None, download_dir_tax=DOWNLOAD_DIR_TAX_NUM):
        self.chrome_execute = chrome_execute
        self.chrome_download = chrome_download
        self.driver = driver if driver is not None else self.restart_driver()        
        self.predict_captcha = predict_captcha if predict_captcha is not None else PredictCaptcha()
        self.search_type = search_type
        self.list_captcha = []
        self.download_dir_tax = download_dir_tax

    def restart_driver(self):
        capabilities = DesiredCapabilities.CHROME
        capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        # chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
        chrome_options.add_argument('--ignore-certificate-errors')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        download_folder = "/Users/truong.vu.1/Documents/Smartpay/source/etl-airflow/data/download"
        prefs = {"profile.default_content_settings.popups": 0,    
                "download.default_directory": self.chrome_download, 
                "download.prompt_for_download": False,
                "download.directory_upgrade": True} if self.chrome_download is not None else None
        chrome_options.add_experimental_option("prefs", prefs) if prefs is not None else None
        chrome_options.add_argument("--headless")
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        # chrome_options.add_argument('--start-maximized');
        # chrome_options.add_argument('--start-fullscreen');
        # chrome_options.add_argument('--window-size=1920,1200');
        # chrome_options.add_argument("force-device-scale-factor=0.75");
        # chrome_options.add_argument("high-dpi-support=0.75");
        
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager("114.0.5735.90").install()),\
                                  options=chrome_options,
                                    desired_capabilities=caps) if self.chrome_execute is None else \
                 webdriver.Chrome(service=Service(self.chrome_execute),\
                                  options=chrome_options,
                                 desired_capabilities=capabilities)
        return self.driver

    def submit_tax_num(self, tax_num):
        
        self.driver.get("https://....")
        ip_tax = MyBrowseUtil.input_element(self.driver, tax_num, None, By.NAME, 'mst') \
                        if self.search_type == "tax_num" else \
                 MyBrowseUtil.input_element(self.driver, tax_num, None, By.NAME, 'cmt')
        
        
        repeat_num = 5
        while repeat_num > 0:
            try:
                body_captcha, captcha_text = None, None
                repeat_num -= 1

                # Get image captcha
                process_browser_log_entry = lambda entry: json.loads(entry['message'])['message']
                browser_log = self.driver.get_log('performance') 
                events = [process_browser_log_entry(entry) for entry in browser_log]
                events = [event for event in events if 'Network.response' in event['method']]
                check_event_captcha = lambda e: True if 'params' in e and 'response' in e['params'] \
                                                            and 'url' in e['params']['response'] \
                                                            and 'captcha' in e['params']['response']['url'] else False

                reqid_captcha = [e for e in events if check_event_captcha(e)][0]['params']['requestId']
                print('reqid_captcha', reqid_captcha)
                body_captcha = self.driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': reqid_captcha})
                imgdata = base64.b64decode(str(body_captcha['body']))
                img_captcha = Image.open(io.BytesIO(imgdata))

                # Input tax and captcha
                captcha_text = self.predict_captcha.predict(img_captcha)['predict']
                print('captcha_text', captcha_text)
                ip_captcha = MyBrowseUtil.input_element(self.driver, captcha_text, None, By.ID, 'captcha')
                time.sleep(1)

                # Click submit
                click_btn = MyBrowseUtil.click_element_wt(self.driver, None, None, By.CLASS_NAME, 'subBtn')
                table_tax = MyBrowseUtil.wait_element(self.driver, None, By.XPATH, '//table[@class="ta_border"]')[0]
                self.list_captcha.append({'img': str(body_captcha['body']), 
                                          'pred': captcha_text, 
                                          'result': 'PASSED'})
                return table_tax
            except selenium.common.exceptions.WebDriverException:
                print('Exception Network.getResponseBody ')
                self.driver.get("https://...")                
                ip_tax = MyBrowseUtil.input_element(self.driver, tax_num, None, By.NAME, 'mst') \
                        if self.search_type == "tax_num" else \
                         MyBrowseUtil.input_element(self.driver, tax_num, None, By.NAME, 'cmt')
                
            except Exception as e:
                self.list_captcha.append({'img': str(body_captcha['body']), 
                                          'pred': captcha_text, 
                                          'result': 'FAILED'}) \
                            if body_captcha is not None and captcha_text is not None else None
                print('submit_tax_num', e)
    

    @staticmethod
    def extract_table_(driver, x_path):
        tbl_tax = MyBrowseUtil.wait_element(driver, None, By.XPATH, x_path)[0]    
        rows = MyBrowseUtil.wait_element(tbl_tax, None, By.XPATH, ".//tr")
        step_try, df = 3, None
        while step_try > 0:
            step_try = step_try - 1

            try:
                cells = rows[0].find_elements(By.XPATH, ".//th | .//td")
                header_parse = [cell.text for cell in cells]

                get_cell = lambda row: [cell for cell in row.find_elements(By.XPATH, ".//td | .//th")]
                rows_parse = [c for c in [get_cell(row) for row in rows] if len(c) > 0]
                df = pd.DataFrame(rows_parse, columns=header_parse)
            except Exception as e:
                print('extract_table_', e, 'Table probably have not been loaded', step_try)
                time.sleep(5)
            
        return df

    @staticmethod
    def convert_df_rec_(df):
        
        dft = df.applymap(lambda x: x.text if x is not None else None)
        part_1 = dict(zip(dft.iloc[:, 0], dft.iloc[:, 1]))
        part_2 = dict(zip(dft.iloc[:, 2], dft.iloc[:, 3]))
        part_3 = dict(zip(dft.iloc[:, 4], dft.iloc[:, 5]))

        part_1.update(part_2)
        part_1.update(part_3)
        
        return part_1
    
    @staticmethod
    def parse_table_html_(html_table):
        # Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(html_table, 'html.parser')

        # Find all <th> tags to extract field names
        fields = [th.text.strip().replace('\n', ' ') for th in soup.find_all('th')]

        # Find all <tr> tags to extract data rows
        data_rows = soup.find_all('tr')[1:]  # Skip the header row

        # Create a list of dictionaries for each row
        data = []
        for row in data_rows:
            row_data = [td.text.strip().replace('\n',' ') for td in row.find_all('td')]
            data.append(dict(zip(fields, row_data)))

        # Convert the data to JSON
        import json
        json_data = json.dumps(data, ensure_ascii=False)

        return json_data
    
    @staticmethod
    def find_link_(x, attr, search_text, tag='a'):
        t = x.find_elements(By.XPATH, f'.//{tag}')
        href = t[0].get_attribute(attr) if len(t) > 0 else None
        return t[0] if href is not None and search_text in href else None
    
    @staticmethod
    def run_js_(driver, tax_num):
        async_script = f"""
            javascript:submitform('{tax_num}')
        """
        try:
            driver.execute_async_script(async_script)
        except:
            pass
        
    def get_table_tax(self):
        df = self.extract_table_(self.driver, '//table[@class="ta_border"]')
        df_list_tax = df.applymap(lambda x: x.text if x is not None else None)
        df_list_tax.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', unidecode(c).lower()).replace("__", "_").replace("__", "_") \
                if c is not None else "empty_col" for c in df_list_tax.columns]
        df_list_tax = df_list_tax[[c for c in df_list_tax.columns if c!= 'empty_col']]
        
        # Get link details
        # dft = df.applymap(lambda x: self.find_link_(x, 'href', 'submit_form') if x is not None else None).stack()
        # link_details = dft[dft.notna()].tolist()
        
        # Remove first and last row
        df_list_tax = df_list_tax.iloc[1:-1,]
        
        return df_list_tax
        
    def full_page_screenshot(self, tax_num):
        
        height = self.driver.execute_script('return document.documentElement.scrollHeight')
        width  = self.driver.execute_script('return document.documentElement.scrollWidth')
        self.driver.set_window_size(width, height)
        # self.driver.execute_script("window.scrollTo(0, 300)")
        time.sleep(1)
        self.driver.save_screenshot(self.download_dir_tax + 'tax_ss_' + str(tax_num).strip() + '.jpg')
        
        
    def get_table_tax_detail(self, tax_num):
        
        # link_detail.click()
        # 
        
        # Get screenshot
        self.full_page_screenshot(tax_num)
        df = self.extract_table_(self.driver, '//table[@class="ta_border"]')        
        records_df = self.convert_df_rec_(df)
        df_final = pd.DataFrame.from_records([records_df])
        df_final.columns = [re.sub(r'[^a-zA-Z0-9_]', '_', unidecode(c).lower()).replace("__", "_").replace("__", "_") \
                if c is not None else "empty_col" for c in df_final.columns]
        df_final = df_final[[c for c in df_final.columns if c!= 'empty_col']]
        return df_final


    # def get_table_tax_extra(self, tax_num):
    #     df = self.extract_table_(self.driver, '//form[@id="loadFrm"]//table')
    #     dft = df.applymap(lambda x: self.find_link_(x, 'link', '.jsp', 'input') if x is not None else None).stack()
    #     link_extras = dft[dft.notna()].tolist()

    #     ls_extra = {}
    #     for ld in link_extras:
    #         url = ld.get_attribute('link') + f'?tin={tax_num}'
    #         html_table = MyBrowseUtil.call_api(self.driver, url, {})
    #         key_url = ld.get_attribute('link').split(".jsp")[0].split("/")[-1]
    #         json_table = self.parse_table_html_(html_table)
    #         print('url link extra', url)
    #         ls_extra.update({key_url: json_table})

    #     return ls_extra
    
    def get_table_tax_extra(self, tax_num): 
        
        # /tcnnt/doanhnghiepchuquan.jsp?tin=2800214981
        # /tcnnt/chinhanh.jsp?tin=2800214981
        # /tcnnt/tructhuoc.jsp?tin=2800214981
        # /tcnnt/daidien.jsp?tin=2800214981
        # /tcnnt/loaithue.jsp?tin=2800214981
        # /tcnnt/nganhkinhdoanh.jsp?tin=2800214981
        
        link_extras = ['/tcnnt/nganhkinhdoanh.jsp', 
                       '/tcnnt/loaithue.jsp']        
        ls_extra = {}
        
        for ld in link_extras:            
            url = ld + f'?tin={tax_num}'
            html_table = MyBrowseUtil.call_api(self.driver, url, {})
            key_url = ld.split(".jsp")[0].split("/")[-1]
            json_table = self.parse_table_html_(html_table)
            print('url link extra', url)
            ls_extra.update({key_url: json_table})
        
        return ls_extra
    
    def get_detail_extra(self, tax_num):
        try:                        
            # Run js
            print(tax_num)
            self.run_js_(self.driver, tax_num)
            time.sleep(2)
            df_detail = self.get_table_tax_detail(tax_num)        
            ls_extra = self.get_table_tax_extra(tax_num)

            # Merge detail & extra
            df_extra = pd.DataFrame.from_records([ls_extra])
            df_detail_extra = pd.concat([df_detail.reset_index(drop=True), df_extra.reset_index(drop=True)], 
                                        axis=1)
            return df_detail_extra

        except Exception as e:
            print(e)
            return None
        
    
    def run_one_tax(self, tax_num):
        repeat_num = 5
        while repeat_num > 0:
            repeat_num -= 1
            try:
                # Get tax
                self.submit_tax_num(tax_num)
                df_tax = self.get_table_tax()

                if len(df_tax) == 0:
                    df_final = pd.DataFrame({'id': [str(tax_num).strip() + '_' + self.search_type[0:3]],
                                             'org_search_key': [str(tax_num).strip()],
                                             'org_search_type': [self.search_type],
                                             'status': ['Non_Exist']})
                else:                    
                    
                    # Get tax detail
                    print('Get tax detail and extra ', datetime.now().strftime("%H:%M:%S"))
                    ls_df_detail_extra = df_tax.iloc[:3,].apply(lambda x:  self.get_detail_extra(x['mst']), 
                                                                axis=1)
                    df_detail_extra = pd.concat(ls_df_detail_extra.to_list())

                    # Merge
                    print('Merge tax list & tax detail & extra', datetime.now().strftime("%H:%M:%S"))
                    df_final = pd.merge(df_tax, df_detail_extra, how = "left", 
                                        left_on="mst", right_on="ma_so_doanh_nghiep")
                    df_final['id'] = str(tax_num).strip() + '_' + self.search_type[0:3] + \
                                                            '_' + df_final['mst']
                    df_final['org_search_key'] = str(tax_num).strip()
                    df_final['org_search_type'] = self.search_type
                    df_final['status'] = 'Exist'

                return df_final
            except Exception as e:
                print('run_one_tax - repeat :', repeat_num, e)
                time.sleep(5)
                self.driver.close()
                self.restart_driver()
                
            return None
                
    def run_list_tax(self, list_tax_num):
        ls_df = []
        for i, tn in enumerate(list_tax_num):
            print('Processing -- ', i, tn, datetime.now().strftime("%H:%M:%S"))
            df_tax_num = self.run_one_tax(str(tn).strip())
            ls_df.append(df_tax_num) if df_tax_num is not None else None
        
        return pd.concat(ls_df) if len(ls_df) > 0 else None

In [59]:
# if df_input is None:
#     class StopExecution(Exception):
#         def _render_traceback_(self):
#             pass

#     print('No exists files')
#     raise StopExecution

In [ ]:
import concurrent.futures
import urllib.request
from IPython.display import clear_output

def run_list(col_search, tax_list):
    crawl_tax = CrawlTax(col_search, CHROME_EXECUTE_EMS)
    df_list = crawl_tax.run_list_tax(tax_list)
    return (df_list, crawl_tax.list_captcha)


while True:
    dir_upload = SFTP_TAX_TO_CRAWL
    df_input, fn_processing = read_sftp(SFTP_BIS3, dir_upload, dir_upload + '/completed/', str, 
                                        ["csv", "xlsx"], None, True, None, {'RETURN_FILE_NAME': True})
    
    # df_input = pd.DataFrame({'tax_num': ['0315097884']})
    
    if df_input is None:
        print('No exists files')
        break
    
    print('---------------- Processing file ------', fn_processing)

    col_search = [c for c in df_input.columns if 'tax_num' in c or 'national_id' in c][0]
    tax_list_all = list(df_input[col_search].unique())
    col_search = 'tax_num' if 'tax_num' in col_search else 'national_id'

    print('col_search, tax_list_all', col_search, tax_list_all[0:5])

    batch_save = 20
    for i in range(0, len(tax_list_all), batch_save):

        # Batch to save
        tax_list = tax_list_all[i:i+batch_save]

        # Running one batch
        ls_batch_res, lsfn_captcha, batch_size = [], [], 20
        with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:

            exe_batch = {executor.submit(run_list, col_search, tax_list[b:b+batch_size]): b \
                                     for b in range(0, len(tax_list), batch_size)}
            for future in concurrent.futures.as_completed(exe_batch):
                batch = exe_batch[future]
                try:
                    data = future.result()
                    ls_batch_res.append(data[0]) if data[0] is not None else None
                    lsfn_captcha += data[1]
                except Exception as e:
                    raise
                    print('ThreadPoolExecutor Exception: ', batch, e)
                else:
                    print('------------- Batch - Len data - Total batch', batch, len(data), len(ls_batch_res))

        # Write list
        print('Save batch list')
        df_list = pd.concat(ls_batch_res).reset_index(drop=True) if len(ls_batch_res)>0 else None
        dt = datetime.now().strftime("%y%m%d%H%M%S")
        file_upload = SFTP_TAX_CRAWL_COMPLETE + f'tax_crawl_{dt}.csv'
        write_sftp(SFTP_BIS3,
                   df_list, 
                   file_upload,
                   option={'csv': True}) if df_list is not None else None
        df_captcha = pd.DataFrame.from_records(lsfn_captcha)

        # Write captcha
        clear_output(wait=True)
        print('Save batch captcha')
        dt = datetime.now().strftime("%y%m%d%H%M%S")
        file_upload = SFTP_TAX_CAPTCHA + f'tax_captcha_{dt}.csv'
        write_sftp(SFTP_BIS3, 
                   df_captcha, 
                   file_upload,
                   option={'csv': True})

#### Transfer files tax to sftp

In [ ]:
# copy tax screenshot to sftp
transfer_sftp(SFTP_BIS3, SFTP_UPLOAD_TAX_NUM, DOWNLOAD_DIR_TAX_NUM, other_params={})

In [ ]:
# move all files to complete
move_dir(DOWNLOAD_DIR_TAX_NUM, DOWNLOAD_DIR_TAX_NUM_COMPLETE)

In [ ]:
# crawl_tax = CrawlTax(CHROME_EXECUTE_EMS)

# df_list = crawl_tax.run_list_tax(tax_list)
# df_list

In [ ]:
# df_list = pd.concat(ls_batch_res).reset_index(drop=True)
# df_list

In [ ]:
# dt = datetime.now().strftime("%y%m%d%H%M%S")
# file_upload = f'/processing/bi_upload/ftp_to_db/tax_crawl/tax_crawl_{dt}.csv'
# write_sftp(SFTP_BIS3, 
#            df_list, 
#            file_upload,
#            option={'csv': True})

In [ ]:
# df_captcha = pd.DataFrame.from_records(lsfn_captcha)
# df_captcha.head(3)

In [ ]:
# dt = datetime.now().strftime("%y%m%d%H%M%S")
# file_upload = f'/processing/bi_upload/ftp_to_db/tax_captcha/tax_captcha_{dt}.csv'
# write_sftp(SFTP_BIS3, 
#            df_captcha, 
#            file_upload,
#            option={'csv': True})

In [ ]:
## BACKUP

In [ ]:
# import pandas as pd

# str_tax = \
# """
# 092089003051
# 381593913
# 024889775
# 023350487
# 381700684
# 142578753
# 351315401
# 077089000860
# 341647711
# """

# list_tax = list(set(str_tax.split("\n")))
# list_tax


# dir_upload = f'/processing/bi_upload/db_to_ftp/tax_crawl/'
# df_list = pd.DataFrame({'national_id': list_tax})
# file_upload = f'{dir_upload}tax_crawl_2.csv'
# write_sftp(SFTP_BIS3,
#            df_list, 
#            file_upload,
#            option={'csv': True})